# LSTM Approach-1

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/datasets/tesla.csv')
data = data.dropna()

In [3]:
# Create features and labels
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

<ipython-input-3-6078241aa6e9>:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Date'] = pd.to_datetime(data['Date'])


In [4]:
# Create lagged features
for lag in range(1, 11):  # Increase the number of lagged features
    data[f'Close_lag_{lag}'] = data['Close'].shift(lag)

In [5]:
# Add more technical indicators as features
data['MA10'] = data['Close'].rolling(window=10).mean()
data['MA50'] = data['Close'].rolling(window=50).mean()
data['RSI'] = data['Close'].diff().rolling(window=14).apply(lambda x: np.mean(np.where(x > 0, x, 0)) / (np.mean(np.abs(x)) + 1e-10), raw=True)
data['EMA10'] = data['Close'].ewm(span=10, adjust=False).mean()
data['EMA50'] = data['Close'].ewm(span=50, adjust=False).mean()

data.dropna(inplace=True)

In [6]:
# Prepare feature and target arrays
features = ['Open', 'High', 'Low', 'Adj Close', 'Volume', 'MA10', 'MA50', 'RSI', 'EMA10', 'EMA50'] + [f'Close_lag_{lag}' for lag in range(1, 11)]
X = data[features]
y = data['Close']

In [7]:
# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [9]:
# Reshape data for LSTM [samples, time steps, features]
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [10]:
# Initialize the LSTM model
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dropout(0.4))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64))
model.add(Dropout(0.4))
model.add(Dense(1))

In [11]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [12]:
# Train the model with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
history = model.fit(X_train_reshaped, y_train, epochs=500, batch_size=16, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/500
86/86 [==============================] - 11s 29ms/step - loss: 41984.4570 - val_loss: 46362.2461
Epoch 2/500
86/86 [==============================] - 1s 13ms/step - loss: 39412.6055 - val_loss: 44535.4102
Epoch 3/500
86/86 [==============================] - 1s 16ms/step - loss: 37602.3281 - val_loss: 42515.0859
Epoch 4/500
86/86 [==============================] - 1s 14ms/step - loss: 35935.9727 - val_loss: 41005.5039
Epoch 5/500
86/86 [==============================] - 2s 23ms/step - loss: 34661.4766 - val_loss: 39591.8633
Epoch 6/500
86/86 [==============================] - 2s 22ms/step - loss: 33432.0430 - val_loss: 38271.2227
Epoch 7/500
86/86 [==============================] - 1s 15ms/step - loss: 32341.5547 - val_loss: 37006.5078
Epoch 8/500
86/86 [==============================] - 1s 14ms/step - loss: 31229.3730 - val_loss: 35805.4023
Epoch 9/500
86/86 [==============================] - 1s 13ms/step - loss: 30197.2246 - val_loss: 34658.0273
Epoch 10/500
86/86 [=======

In [13]:
# Make predictions on the testing set
y_pred = model.predict(X_test_reshaped)

14/14 [==============================] - 1s 5ms/step


In [14]:
# Flatten the predictions
y_pred = y_pred.flatten()

In [15]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [16]:
# Calculate a custom accuracy metric
def custom_accuracy(y_true, y_pred, tolerance=0.05):
    return np.mean(np.abs((y_true - y_pred) / y_true) <= tolerance)

accuracy = custom_accuracy(y_test, y_pred)

In [17]:
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R2): {r2}')
print(f'Custom Accuracy: {accuracy * 100:.2f}%')

Mean Absolute Error (MAE): 2.457856670133577
Mean Squared Error (MSE): 21.226325370592562
R-squared (R2): 0.9984027705763768
Custom Accuracy: 87.65%


In [18]:
# Make a prediction on new data
new_data = pd.DataFrame({
    'Open': [300],
    'High': [310],
    'Low': [295],
    'Adj Close': [305],
    'Volume': [1000000],
    'MA10': [305],
    'MA50': [310],
    'RSI': [0.01],
    'EMA10': [305],
    'EMA50': [310],
    'Close_lag_1': [298],
    'Close_lag_2': [299],
    'Close_lag_3': [297],
    'Close_lag_4': [296],
    'Close_lag_5': [295],
    'Close_lag_6': [294],
    'Close_lag_7': [293],
    'Close_lag_8': [292],
    'Close_lag_9': [291],
    'Close_lag_10': [290]
})

In [19]:
new_data_scaled = scaler.transform(new_data)
new_data_reshaped = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))
new_prediction = model.predict(new_data_reshaped)
print(f'Predicted Close Price: {new_prediction[0][0]}')

1/1 [==============================] - 0s 25ms/step
Predicted Close Price: 300.8011779785156


# LSTM Approach-2

In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [21]:
# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/datasets/tesla.csv')
data = data.dropna()

In [22]:
# Create features and labels
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

<ipython-input-22-6078241aa6e9>:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Date'] = pd.to_datetime(data['Date'])


In [23]:
# Create lagged features
for lag in range(1, 11):  # Increase the number of lagged features
    data[f'Close_lag_{lag}'] = data['Close'].shift(lag)

In [24]:
# Add more technical indicators as features
data['MA10'] = data['Close'].rolling(window=10).mean()
data['MA50'] = data['Close'].rolling(window=50).mean()
data['RSI'] = data['Close'].diff().rolling(window=14).apply(lambda x: np.mean(np.where(x > 0, x, 0)) / (np.mean(np.abs(x)) + 1e-10), raw=True)

data.dropna(inplace=True)

In [25]:
# Prepare feature and target arrays
features = ['Open', 'High', 'Low', 'Adj Close', 'Volume', 'MA10', 'MA50', 'RSI'] + [f'Close_lag_{lag}' for lag in range(1, 11)]
X = data[features]
y = data['Close']

In [26]:
# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [27]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [28]:
# Reshape data for LSTM [samples, time steps, features]
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [29]:
# Initialize the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dropout(0.3))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(64))
model.add(Dropout(0.3))
model.add(Dense(1))

In [30]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [31]:
# Train the model with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train_reshaped, y_train, epochs=300, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/300
43/43 [==============================] - 7s 37ms/step - loss: 43540.7773 - val_loss: 48891.8438
Epoch 2/300
43/43 [==============================] - 1s 14ms/step - loss: 41042.9844 - val_loss: 46263.5469
Epoch 3/300
43/43 [==============================] - 1s 14ms/step - loss: 39598.2578 - val_loss: 45119.1328
Epoch 4/300
43/43 [==============================] - 1s 15ms/step - loss: 38682.5234 - val_loss: 44126.2812
Epoch 5/300
43/43 [==============================] - 1s 13ms/step - loss: 37780.7422 - val_loss: 43148.5547
Epoch 6/300
43/43 [==============================] - 1s 14ms/step - loss: 36815.5273 - val_loss: 42086.6055
Epoch 7/300
43/43 [==============================] - 1s 14ms/step - loss: 35866.4922 - val_loss: 41170.3750
Epoch 8/300
43/43 [==============================] - 1s 12ms/step - loss: 35080.2695 - val_loss: 40390.0430
Epoch 9/300
43/43 [==============================] - 0s 9ms/step - loss: 34409.3008 - val_loss: 39640.0859
Epoch 10/300
43/43 [=========

In [32]:
# Make predictions on the testing set
y_pred = model.predict(X_test_reshaped)

14/14 [==============================] - 1s 2ms/step


In [33]:
# Flatten the predictions
y_pred = y_pred.flatten()

In [34]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [35]:
# Calculate a custom accuracy metric
def custom_accuracy(y_true, y_pred, tolerance=0.05):
    return np.mean(np.abs((y_true - y_pred) / y_true) <= tolerance)

accuracy = custom_accuracy(y_test, y_pred)

In [36]:
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R2): {r2}')
print(f'Custom Accuracy: {accuracy * 100:.2f}%')

Mean Absolute Error (MAE): 5.790148516981717
Mean Squared Error (MSE): 63.5373168058954
R-squared (R2): 0.9952189712477958
Custom Accuracy: 89.74%


In [37]:
# Make a prediction on new data
new_data = pd.DataFrame({
    'Open': [300],
    'High': [310],
    'Low': [295],
    'Adj Close': [305],
    'Volume': [1000000],
    'MA10': [305],
    'MA50': [310],
    'RSI': [0.01],
    'Close_lag_1': [298],
    'Close_lag_2': [299],
    'Close_lag_3': [297],
    'Close_lag_4': [296],
    'Close_lag_5': [295],
    'Close_lag_6': [294],
    'Close_lag_7': [293],
    'Close_lag_8': [292],
    'Close_lag_9': [291],
    'Close_lag_10': [290]
})

In [38]:
new_data_scaled = scaler.transform(new_data)
new_data_reshaped = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))
new_prediction = model.predict(new_data_reshaped)
print(f'Predicted Close Price: {new_prediction[0][0]}')

1/1 [==============================] - 0s 23ms/step
Predicted Close Price: 286.53228759765625
